In [259]:
import pandas as pd
from pathlib import Path
import json
import seaborn as sns
import requests
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [260]:
WEATHER_LOAD_DIR = Path(
    Path.cwd().parent, "data.nosync", "combined_data", "combined_weather.parquet"
)
WEATHER_MAPPING = Path(
    Path.cwd().parent, "data_information", "weather_location_mapping.json"
)

In [261]:
weather_df = pd.read_parquet(WEATHER_LOAD_DIR)

In [262]:
weather_df = weather_df.drop(
    list(weather_df.keys())[-1], axis=1
)  # weird empty column at the end of df

In [263]:
weather_df.head()

Forecast Date Vintage Date Vintage Station ID  Max Temp  Min Temp  \
0    2011-03-02   2011-02-27  Actual        ALB        37        22   
1    2011-03-02   2011-02-27  Actual        ART        39        19   
2    2011-03-02   2011-02-27  Actual        BGM        34        26   
3    2011-03-02   2011-02-27  Actual        BUF        38        26   
4    2011-03-02   2011-02-27  Actual        ELM        39        27   

   Max Wet Bulb  Min Wet Bulb  
0            32          21.0  
1            35          18.0  
2            31          25.0  
3            35          26.0  
4            35          26.0

# Mapping Station ID to location

In order to map the correct weather station to the energy demand data, we need to map it to a location.
I've manually looked up where each of the stations are and mapped them in data information/weather_location_map.json

In [264]:
[x for x in weather_df["Station ID"].unique()]

['ALB',
 'ART',
 'BGM',
 'BUF',
 'ELM',
 'HPN',
 'ISP',
 'JFK',
 'LGA',
 'MSS',
 'MSV',
 'PLB',
 'POU',
 'ROC',
 'SWF',
 'SYR',
 'UCA',
 'ELZ',
 'FOK',
 'FRG',
 'GFL',
 'IAG',
 'ITH',
 'NYC',
 'SLK']

In [265]:
weather_mapping = json.loads(Path(WEATHER_MAPPING).read_text(encoding="UTF-8"))
weather_mapping

{'ALB': ['42.652580', '-73.756233'],
 'ART': ['43.974785', '-75.910759'],
 'BGM': ['42.098843', '-75.920647'],
 'BUF': ['42.880230', '-78.878738'],
 'ELM': ['42.084972', '-76.798553'],
 'HPN': ['41.033985', '-73.762909'],
 'ISP': ['40.732071', '-73.186180'],
 'JFK': ['40.641766', '-73.780968'],
 'LGA': ['40.7769', '-73.8740'],
 'MSS': ['44.933910', '-74.897629'],
 'MSV': ['41.6556', '-74.6893'],
 'PLB': ['44.699764', '-73.471428'],
 'POU': ['41.708290', '-73.923912'],
 'ROC': ['43.161030', '-77.610924'],
 'SWF': ['41.512177', '-74.018326'],
 'SYR': ['43.088947', '-76.154480'],
 'UCA': ['43.227978', '-75.484924']}

Note we have some cities in our dataframe which we dont have location info for. This is due to the information on translating the area codes to actual places isn't complete. 
We'll drop any cities we dont have location data for. 

In [266]:
cities_with_location_data = set(weather_mapping.keys())
cities_in_weather_df = set(weather_df["Station ID"].unique())
cities_intersection = list(cities_in_weather_df.intersection(cities_with_location_data))

In [267]:
print(
    f"""{weather_df.shape[0] - 
      weather_df[weather_df['Station ID'].isin(cities_intersection)].shape[0]} rows removed"""
)
weather_df = weather_df[weather_df["Station ID"].isin(cities_intersection)].reset_index(
    drop=True
)

82120 rows removed


In [268]:
weather_df["lat_lon"] = weather_df["Station ID"].map(weather_mapping)

In [269]:
# splitting the lat lon column into two seperate columns
lat_lon_df = pd.DataFrame(weather_df["lat_lon"].values.tolist()).rename(
    columns={0: "lat", 1: "lon"}
)
weather_df = pd.concat([weather_df, lat_lon_df], axis=1).drop("lat_lon", axis=1, errors = 'ignore')
weather_df['lat'] = weather_df['lat'].astype(float)
weather_df['lon'] = weather_df['lon'].astype(float)
weather_df['Max Wet Bulb'] = weather_df['Max Wet Bulb'].astype(float)

In [270]:
weather_df.head()

Forecast Date Vintage Date Vintage Station ID  Max Temp  Min Temp  \
0    2011-03-02   2011-02-27  Actual        ALB        37        22   
1    2011-03-02   2011-02-27  Actual        ART        39        19   
2    2011-03-02   2011-02-27  Actual        BGM        34        26   
3    2011-03-02   2011-02-27  Actual        BUF        38        26   
4    2011-03-02   2011-02-27  Actual        ELM        39        27   

   Max Wet Bulb  Min Wet Bulb        lat        lon  
0          32.0          21.0  42.652580 -73.756233  
1          35.0          18.0  43.974785 -75.910759  
2          31.0          25.0  42.098843 -75.920647  
3          35.0          26.0  42.880230 -78.878738  
4          35.0          26.0  42.084972 -76.798553

# Visualizing

Where are out stations located?

In [271]:
station_locations = weather_df[['Station ID', 'lat', 'lon']].groupby('Station ID', as_index = False).mean()

In [274]:
fig = go.Figure(data=go.Scattergeo(
        lon = station_locations['lon'], 
        lat = station_locations['lat'],
        text = station_locations['Station ID'],
        mode = 'markers',
        ))


# Adding state lines - from 
#https://stackoverflow.com/questions/71230623/how-to-show-state-boundaries-only-for-us-on-a-world-scatter-geo
states_geojson = requests.get(
    "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_1_states_provinces_lines.geojson"
).json()


fig = fig.add_trace(
    go.Scattergeo(
        lat=[
            v
            for sub in [
                np.array(f["geometry"]["coordinates"])[:, 1].tolist() + [None]
                for f in states_geojson["features"]
            ]
            for v in sub
        ],
        lon=[
            v
            for sub in [
                np.array(f["geometry"]["coordinates"])[:, 0].tolist() + [None]
                for f in states_geojson["features"]
            ]
            for v in sub
        ],
        line_color="brown",
        line_width=1,
        mode="lines",
        showlegend=False,
    )
)
fig.update_layout(
        title = 'Weather Station Locations',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True
        ))

lat_foc = 43.2
lon_foc = -75.5
fig.update_layout(
        geo = dict(
            projection_scale=5, #this is kind of like zoom
            center=dict(lat=lat_foc, lon=lon_foc), # this will center on the point
        ))

fig.show()

They all appear to be within NY state boundaries which is reassuring!